In [1]:
def list_stats(nummies):
  nummies.sort()
  midull = len(nummies)//2
  medianna = (nummies[midull - 1] + nummies[midull])/2
  mean = sum(nummies)/len(nummies)
  return medianna, mean

# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # Run your function with the first example in the question.
  m = list_stats([1.3, 2.4, 20.6, 0.95, 3.1, 2.7])
  print(m)

  # Run your function with the second example in the question
  m = list_stats([1.5])
  print(m)

(2.55, 5.175000000000001)
(1.5, 1.5)


In [2]:
import numpy as np
import statistics
import time

def time_stat(func, size, ntrials):
  total = 0
  for x in range(ntrials):
    data = np.random.rand(size)
    start = time.perf_counter()
    res = func(data)
    total += time.perf_counter() - start
  return total/ntrials

if __name__ == '__main__':
  print('{:.6f}s for statistics.mean'.format(time_stat(statistics.mean, 10**5, 10)))
  print('{:.6f}s for np.mean'.format(time_stat(np.mean, 10**5, 1000)))


0.248965s for statistics.mean
0.000101s for np.mean


In [3]:
import time, numpy as np
from astropy.io import fits
def median_FITS(filez):
  start = time.time()
  lizt = []
  for ek in filez:
    hdulist = fits.open(ek)
    lizt.append(hdulist[0].data)
    hdulist.close()
  stacking = np.dstack(lizt)
  median = np.median(stacking, axis = 2)
  memory = stacking.nbytes
  memory /= 1024
  stop = time.time() - start
  return median, stop, memory


# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # Run your function with first example in the question.
  result = median_FITS(['image0.fits', 'image1.fits'])
  print(result[0][100, 100], result[1], result[2])
  
  # Run your function with second example in the question.
  result = median_FITS(['image{}.fits'.format(str(i)) for i in range(11)])
  print(result[0][100, 100], result[1], result[2])

FileNotFoundError: [Errno 2] No such file or directory: 'image0.fits'

In [4]:
import numpy as np

def median_bins(nummies, ek):
  mean = np.mean(nummies)
  std = np.std(nummies)
  binuno = 0
  bins = np.zeros(ek)
  bin_width = 2*std/ek
  for num in nummies:
    if num < mean - std:
      binuno += 1
    elif num < mean + std:
      bin = int((num - (mean - std))/bin_width)
      bins[bin] += 1
  return mean, std, binuno, bins


def median_approx(nummies, ek):
  mean, std, binuno, bins = median_bins(nummies, ek)
  x = len(nummies)
  midull = (x + 1)/2
  count = binuno
  for b, bincount in enumerate(bins):
    count += bincount
    if count >= midull:
      break

  width = 2*std/ek
  median = mean - std + width*(b + 0.5)
  return median

print(median_bins([1, 1, 3, 2, 2, 6], 3))
print(median_approx([1,1,3,2,2,6], 3))

(2.5, 1.707825127659933, 0, array([2., 3., 0.]))
2.5


In [5]:
# Import the running_stats function
from helper import running_stats
import time, numpy as np
from astropy.io import fits
# Write your median_bins_fits and median_approx_fits here:
def median_bins_fits(filez, ek):
  mean, std = running_stats(filez)
  dim = mean.shape
  binuno = np.zeros(dim)
  bins = np.zeros((dim[0], dim[1], ek))
  width = 2*std/ek
  for pg in filez:
    hdulist = fits.open(pg)
    data = hdulist[0].data
    for r in range(dim[0]):
      for c in range(dim[1]):
        val = data[r,c]
        meanman = mean[r,c]
        deviate = std[r,c]
        if val < meanman - deviate:
          binuno[r,c] += 1
        elif val >= meanman - deviate and val < meanman + deviate:
          bin = int((val - (meanman - deviate))/width[r,c])
          bins[r,c,bin] += 1
  return mean, std, binuno, bins

def median_approx_fits(filez, ek):
  mean, std, binuno, bins = median_bins_fits(filez, ek)
  dim = mean.shape
  a = len(filez)
  midull = (a + 1)/2
  width = 2*std/ek
  median = np.zeros(dim)
  for r in range(dim[0]):
    for c in range(dim[1]):
      count = binuno[r,c]
      for b, bincount in enumerate(bins[r,c]):
        count += bincount
        if count >= midull:
          break
      median[r,c] = mean[r,c] - std[r,c] + width[r,c]*(b+0.5)    
  return median

# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # Run your function with examples from the question.
  mean, std, binuno, bins = median_bins_fits(['image0.fits', 'image1.fits', 'image2.fits'], 5)
  median = median_approx_fits(['image0.fits', 'image1.fits', 'image2.fits'], 5)

ModuleNotFoundError: No module named 'helper'